## Execute Task A Stunt

### Setup and Connect

In [26]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np
import matplotlib.pyplot as plt

class RobotControl():
    # Initialize Function
    def __init__(self, ble):
        self.ble = ble
        self.notifiers_on = False
        self.TOF_HISTORY_LEN = 60
        self.IMU_HISTORY_LEN = 300
        self.calibration_factor = 0

        # A variable to store the latest sensor value
        self.latest_tof_front_reading = None
        
        # A list to store the history of all the sensor values
        # Each item in the list is a tuple (value, time)
        # WARNING: The list could grow really fast; you need to deal with this accordingly.
        self.tof1_readings = [] # TODO change these to a bounded queue
        self.tof2_readings = []
        self.imu_readings = []
        self.bot_pid_debug = []
        
        # A variable to store the latest imu reading
        self.latest_imu_reading = None
        
        # Activate notifications (if required)
        self.setup_notify()
    
    # A function to activate various notifications (if required)
    def setup_notify(self):
        # Code to setup various notify events
        # Ex:
        ble.start_notify(ble.uuid['RX_TOF1'], self.tof1_callback_handler)
        ble.start_notify(ble.uuid['RX_TOF2'], self.tof2_callback_handler)
        ble.start_notify(ble.uuid['RX_IMU'], self.imu_callback_handler)
        ble.start_notify(ble.uuid['RX_MOTOR_PID'], self.motor_pid_callback_handler)
        self.notifiers_on = True

    def stop_notify(self):
        ble.stop_notify(ble.uuid['RX_TOF1'])
        ble.stop_notify(ble.uuid['RX_TOF2'])
        ble.stop_notify(ble.uuid['RX_IMU'])
        ble.stop_notify(ble.uuid['RX_MOTOR_PID'])
        self.notifiers_on = False
    
    def tof1_callback_handler(self, uuid, byte_array):
        self.tof1_readings.append( self.ble.bytearray_to_float(byte_array) )

    def tof2_callback_handler(self, uuid, byte_array):
        self.tof2_readings.append( ( self.ble.bytearray_to_float(byte_array), time.time() ) )
    
    def imu_callback_handler(self, uuid, byte_array):
        self.imu_readings.append( (self.ble.bytearray_to_string(byte_array), time.time()) )

    def motor_pid_callback_handler(self, uuid, byte_array):
        self.bot_pid_debug.append( self.ble.bytearray_to_float(byte_array) )

    # An example function to fetch the front TOF sensor reading
    # Here we assume RX_TOF1 is a valid UUID defined in connection.yaml and
    # in the Arduino code as well
    def get_front_tof(self):
        ble.send_command(CMD.GET_FRONT_TOF, None)
        return self.tof1_readings[-1]

    
    # An example function to fetch the IMU readings as a string
    # Here we assume RX_IMU is a valid UUID defined in connection.yaml and
    # in the Arduino code as wellt
    # def get_imu(self):
    #     self.latest_imu_reading = self.ble.receive_string(self.ble.uuid['RX_IMU'])
    #     pass
    
    def start_pid(self, setpoint):
        ble.send_command(CMD.START_PID, setpoint)

    def stop_pid(self):
        ble.send_command(CMD.STOP_PID, None)

    # A function to instruct the robot to move forward
    def move_forward(self, speed):
        ble.send_command(CMD.MOVE_FORWARD, speed)
    
    def move_backward(self, speed):
        ble.send_command(CMD.MOVE_BACKWARD, speed)

    def set_motor_calibration(self, new_val):
        ble.send_command(CMD.SET_MOTOR_CALIB, new_val)
        self.calibration_factor = new_val
    
    # A function to stop robot motion
    def stop(self):
        ble.send_command(CMD.STOP, None)

    def start_data_collection(self):
        ble.send_command(CMD.START_DATA_COLLECTION, None)
    
    def stop_data_collection(self):
        temp_notif = self.notifiers_on
        if not temp_notif:
            self.setup_notify()
        ble.send_command(CMD.STOP_DATA_COLLECTION, None)
        for i in range(self.TOF_HISTORY_LEN):
            ble.send_command(CMD.GET_TOF1_DATA, i)
            ble.send_command(CMD.GET_PID_DATA, i)
        if not temp_notif:
            self.stop_notify()

    """
    Step response, but hard break at step_stop
    """
    def start_step_response(self, step_stop):
        ble.send_command(CMD.START_STEP_RESPONSE, step_stop)
    
    def stop_step_response(self):
        ble.send_command(CMD.STOP_STEP_RESPONSE, None)

    def do_stunt(self, stunt_setpoint, stunt_pid_proportional):
        ble.send_command(CMD.START_STUNT, f"{stunt_setpoint}|{stunt_pid_proportional}")

    def sudden_flip(self, raw_power, scale=False):
        if scale:
            ble.send_command(CMD.SUDDEN_FLIP, raw_power * 255 / self.calibration_factor)
        else:
            ble.send_command(CMD.SUDDEN_FLIP, raw_power)
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Define Robot Controller Class

In [27]:
LOG.propagate = False
ble = get_ble_controller()
ble.connect()

2022-03-27 17:38:56,113 | INFO     |: Looking for Artemis Nano Peripheral Device: C0:07:21:8D:B3:44
2022-03-27 17:38:59,162 | INFO     |: Connected to C0:07:21:8D:B3:44


### Run Stunt

In [28]:
bot = RobotControl(ble)

In [32]:

# bot.set_motor_calibration(1.8)
# bot.start_data_collection()
# bot.start_pid(0.1)
# time.sleep(1)
# bot.stop_pid()
# bot.move_backward(255)
# time.sleep(1)
# bot.stop()
# bot.stop_data_collection()

bot.set_motor_calibration(1.8)
bot.start_data_collection()
bot.move_forward(255)
time.sleep(1)
bot.sudden_flip(255, scale=False)
time.sleep(1)
bot.stop()
bot.stop_data_collection()

In [33]:
print(f"TOF: {bot.tof1_readings}")
print(f"PID: {bot.bot_pid_debug}")

TOF: [505.0, 55786.0, 534.0, 56267.0, 596.0, 56321.0, 492.0, 56419.0, 572.0, 56475.0, 605.0, 56531.0, 667.0, 56587.0, 668.0, 56643.0, 678.0, 56698.0, 539.0, 56795.0, 492.0, 56893.0, 438.0, 56998.0, 463.0, 57096.0, 430.0, 57194.0, 423.0, 57292.0, 452.0, 57389.0, 535.0, 57488.0, 734.0, 57587.0, 748.0, 57686.0, 1668.0, 57782.0, 1869.0, 57882.0, 2113.0, 57981.0, 2282.0, 58077.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2556.0, 78314.0, 2575.0, 78411.0, 2548.0, 78510.0, 2477.0, 78610.0, 2389.0, 78705.0, 2240.0, 78805.0, 2110.0, 78903.0, 1951.0, 79001.0, 1768.0, 79099.0, 1574.0, 79196.0, 1350.0, 79298.0, 1115.0, 79393.0, 848.0, 79501.0, 633.0, 79602.0, 490.0, 79699.0, 